In [50]:
from collections import namedtuple
import re

In [55]:
py_sig_func = re.compile(
    r'''^ \s* (?: \((.*)\)       # optional: arguments
              (?:\s* -> \s* (.*))?  #           return annotation
              )? $                   # and nothing more
          ''', re.VERBOSE)

class IrregularExpression(object):
    FakeMatch = namedtuple('FakeMatch', 'groups')

    def match(self, s):
        # explicit_modname
        explicit_modname = None
        if "::" in s:
            explicit_modname, s = s.split("::")
        pieces = []
        piece = ''
        num_open = 0
        i = 0
        for c in s:
            if c.isspace() or c == '(':
                break
            if num_open == 0 and c == '.':
                pieces.append(piece)
                piece = ''
            else:
                if c == '[':
                    num_open += 1
                elif c == ']':
                    num_open -= 1
                piece += c
            i += 1
        pieces.append(piece)
        remaining = s[i:]
        r_groups = py_sig_func.match(remaining).groups()
        path = '.'.join(pieces[:-1]) or None
        base = pieces[-1]
        groups = (
            explicit_modname,
            path,
            base,
        ) + r_groups
        return self.FakeMatch(lambda: groups)

py_ext_sig_re = IrregularExpression()

In [57]:
#: extended signature RE: with explicit module name separated by ::
py_ext_sig_re_orig = re.compile(
    r'''^ ([\w.]+::)?            # explicit module name
          ([\w.]+\.)?            # module and/or class name(s)
          (\w+)  \s*             # thing name
          (?: \((.*)\)           # optional: arguments
           (?:\s* -> \s* (.*))?  #           return annotation
          )? $                   # and nothing more
          ''', re.VERBOSE)

In [60]:
names = [
    "a.b.Name",
    "a.b.Name[Hello]",
    "x::a.b.Name[Stuff.Bye].Yawrs[x[1.2]](self, x, y) -> None",
]

for name in names:
    print(py_ext_sig_re.match(name).groups())
    print(py_ext_sig_re_orig.match(name.replace('[', '_').replace(']', '_')).groups())
    print("---")

(None, 'a.b', 'Name', None, None)
(None, 'a.b.', 'Name', None, None)
---
(None, 'a.b', 'Name[Hello]', None, None)
(None, 'a.b.', 'Name_Hello_', None, None)
---
('x', 'a.b.Name[Stuff.Bye]', 'Yawrs[x[1.2]]', 'self, x, y', 'None')
('x::', 'a.b.Name_Stuff.Bye_.Yawrs_x_1.', '2__', 'self, x, y', 'None')
---
